In [29]:
# Install packages as needed
#%pip install openpyxl
#%pip install pandas 
#%pip install plotly
#%pip install nbformat
#%pip install --upgrade ipython

In [30]:
#import libraries
import pandas as pd
import json
import os
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import locale
import numpy as np
import re

Extract

In [31]:
###EXTRACT###
# Read Transaction Excel data into a Pandas dataframe
file = r'C:\Users\DIM6\OneDrive - PGE\[Work] [Shared]\CXI Projects\20200122 03 CC Fact Sheet\2022\2022 Customer Transactions Map - Publication Sheets.xlsx'
sheet_name = 'BarGraphs'

# Read the specific sheet from the Excel file
df_raw = pd.read_excel(file, sheet_name=sheet_name)
print(df_raw.head())

  Root Level Level 1 Level 2a Level 2  Value
0        NaN     NaN      NaN     NaN    NaN
1        NaN     NaN      NaN     NaN    NaN
2        NaN     NaN      NaN     NaN    NaN
3        NaN     NaN      NaN     NaN    NaN
4        NaN     NaN      NaN     NaN    NaN


In [32]:
#Column Headers
Root = 'Root Level'
L1 = "Level 1" 
L2 = "Level 2"
Value = "Value"

In [33]:
 #Remove columns not needed for sunburst
df = df_raw[[Root, L1, L2, Value ]]

# convert cells in column "Value" to numeric values
df[Value] = pd.to_numeric(df[Value], errors='coerce')

# Remove rows with all NaN values
filtered_df = df.dropna(how='all')

# Filter out rows with NaN or blank values in the "Value" column
df = filtered_df[(filtered_df[Value].notna()) & (filtered_df[Value] != '')]

## Replace NaN with None for plotly sunburst chart
df = df.where(pd.notna(df), "")

print(df.head())

          Root Level   Level 1   Level 2       Value
43               NPC  Payments  Payments   1770000.0
49              Mail  Payments  Payments  11200000.0
58  Other Electronic  Payments  Payments  10393000.0
59  Other Electronic  Payments  Payments   2735000.0
60  Other Electronic  Payments  Payments    637000.0


C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_6136\3408908783.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
def fill_empty_L2(row):
    if row[L2] == "":
        return row[L1]
    else:
        return row[L2]

# Fill empty strings in L2 with corresponding L1 values
df[L2] = df.apply(fill_empty_L2, axis=1)

In [35]:
# Remove "-" and whitespace
#df[Root] = df[Root].str.replace(r"\s*-\s*", "").str.strip().astype(str)
df[L1] = df[L1].str.replace(r"\s*-\s*", "").str.strip().astype(str)
df[L2] = df[L2].str.replace(r"\s*-\s*", "").str.strip().astype(str)

#Convert L2 to proper case
df[L2] = df[L2].str.title()

C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_6136\1283686359.py:3: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_6136\1283686359.py:4: FutureWarning:

The default value of regex will change from True to False in a future version.



In [36]:
# Splice a new df
df = df.copy()

print(df.head())

          Root Level   Level 1   Level 2       Value
43               NPC  Payments  Payments   1770000.0
49              Mail  Payments  Payments  11200000.0
58  Other Electronic  Payments  Payments  10393000.0
59  Other Electronic  Payments  Payments   2735000.0
60  Other Electronic  Payments  Payments    637000.0


In [37]:
# Convert dataframe to a hierarchical dictionary
def make_hierarchy_dict(df, path=[], value_col=Value, threshold=10):
    if len(path) == df.columns.nlevels - 1:
        total = int(df[value_col].sum())
        if total < threshold:
            return None
        else:
            return total
    else:
        level = df.columns[len(path)]
        subdict = {}
        for key in df[level].dropna().unique():
            subdf = df[df[level] == key].drop(columns=[level])
            subhierarchy_dict = make_hierarchy_dict(subdf, path + [key], value_col, threshold)
            if subhierarchy_dict is not None:
                subdict[key] = subhierarchy_dict
        if len(subdict) == 0:
            return None
        else:
            return subdict

hierarchy_dict = make_hierarchy_dict(df.set_index(list(df.columns[:3])), value_col=Value, threshold=10)

print(hierarchy_dict)

177218424


In [38]:
# Convert hierarchical dictionary to Plotly-compatible JSON
def make_plotly_sunburst(hierarchy_dict, name=""):
    if isinstance(hierarchy_dict, dict):
        children = []
        for key, value in hierarchy_dict.items():
            children.append(make_plotly_sunburst(value, key))
        return {
            "name": name,
            "children": children
        }
    else:
        return {
            "label": name,
            "value": hierarchy_dict
        }

In [39]:
# Write JSON data to a file
plotly_data = make_plotly_sunburst(hierarchy_dict)

with open("plotly_data.json", "w") as f:
    json.dump(plotly_data, f)

    print(plotly_data)

{'label': '', 'value': 177218424}


In [40]:
# Write dataframe to an Excel file for QC
with pd.ExcelWriter('dataframe-bargraph.xlsx') as writer:
    df.to_excel(writer, sheet_name='DataFrame', index=False)

In [41]:
#Group the data by Root Level and sum the values
grouped_data = df.groupby([Root])[Value].sum().reset_index()

# Sort the grouped_data by Value column in descending order
grouped_data = grouped_data.sort_values(by=Value, ascending=False)

# Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood', 'Call Center - IVR': 'deepskyblue','Call Center - Other':'lightblue', \
              'Call Center - Live Agent':'Turquoise', 'NPC': 'gold'}

# Create the bar chart trace
bar_trace = go.Bar(
    x=grouped_data[Root],
    y=grouped_data[Value],
    marker=dict(color=[color_map.get(root_level, 'grey') for root_level in grouped_data[Root]]),
    text=[f"{value:,}" for value in grouped_data[Value]],
    textposition='auto'
)

# Create the bar chart layout
bar_layout = go.Layout(
    title='Total Customer Interactions by Transaction Channel',
    xaxis=dict(title='Channel'),
    yaxis=dict(title='Total Interactions'),
    legend=dict(title=dict(text='Transaction Channels'))
)

# Create the bar chart figure
bar_fig_root = go.Figure(data=[bar_trace], layout=bar_layout)

# Display the bar chart
bar_fig_root.show()


In [42]:
# Export the chart as an HTML file
#pyo.plot(fig, filename='Trxn_sunburst_chart.html', auto_open=True)